In [1]:
from google.colab import drive

drive.mount('/drive')

Mounted at /drive


In [2]:
#!git clone https://github.com/jthickstun/watermark
!pip install -q transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.1 MB/s eta 0:00:00


In [3]:
cd '/drive/My Drive/Colab Notebooks/ISO/'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO


In [4]:
cd kuditipudi-watermark

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/kuditipudi-watermark


In [5]:
!pip install -q -r new_requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.5 MB/s eta 0:00:00


In [6]:
cd demo

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/kuditipudi-watermark/demo


In [7]:
import os, argparse

import torch
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from mersenne import mersenne_rng
from generate import generate_shift

In [8]:
checkpoint = 'facebook/opt-1.3b'

model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/653 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [9]:
cd '../../instances'

/drive/.shortcut-targets-by-id/1k2JFo63MNzskIgAHpOe33d-BrKTiK7mE/ISO/instances


In [10]:
import string
p = set(string.printable)

In [11]:
filename = 'kuditipudi_no_attack.json'

df = pd.read_json('c4_selection.json', lines=True)

with open(filename, "r+" if os.path.exists(filename) else "w+") as f:
    with torch.no_grad():
        for i in tqdm(range(len(f.readlines()), len(df["input"]))):
            line = df["input"][i]
            tokenized_input = tokenizer(line, return_tensors='pt').to(model.device)

            output_tokens = generate_shift(model=model, prompt=tokenized_input["input_ids"], vocab_size=len(tokenizer), n=200, m=200, key=42)[0]

            # if decoder only model, then we need to isolate the
            # newly generated tokens as only those are watermarked, the input/prompt is not

            output_text = tokenizer.decode(output_tokens[51:], skip_special_tokens=True)

            _line = ''.join(filter(lambda x: x in p, line)).replace('\n', '\\n').replace('"', '\\"')
            _output_text = ''.join(filter(lambda x: x in p, output_text)).replace('\n', '\\n').replace('"', '\\"')

            f.write(f'{{"input": "{_line}", "continuation": "{_output_text}"}}\n')

100%|██████████| 66/66 [31:58<00:00, 29.07s/it]
